In [15]:
from bs4 import BeautifulSoup
import json
import requests
from selenium import webdriver

ModuleNotFoundError: No module named 'selenium'

In [11]:
def get_html(url):
    """
    функция достаёт html по url
    """
    ret = None
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
        r = requests.get(url, headers=headers, verify=False)
        ret = r.text
    except Exception as e:
        print(str(e))

    return ret

In [12]:
url = 'http://dmk.su/truby-i-otvody-jshop/truby-bolshogo-diametra/truba-820kh14-mm-i-vyshe-l-6m-12m'
heml = get_html(url)
soup = BeautifulSoup(html, 'lxml')
product_div = soup.find('div', {'class': 'tab-cont active'})

C:\Users\aobor\Anaconda3\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://dmk.su/truby-i-otvody-jshop/truby-bolshogo-diametra/truba-820kh14-mm-i-vyshe-l-6m-12m" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [14]:
print(soup)

<html><body><p>http://dmk.su/truby-i-otvody-jshop/truby-bolshogo-diametra/truba-820kh14-mm-i-vyshe-l-6m-12m</p></body></html>


In [ ]:
p_data = product_div.get_text(separator='\n').strip().split('\n')
h1 = soup.find('h1', {'class': '456'}).text.strip().split()

In [ ]:
if len(h1) == 6:
    product_type = h1[0] + ' ' + h1[1]
    try:
        diameter = float(h1[2])
    except:
        diameter = None
    try:
        wall = float(h1[4].replace(',', '.'))
    except:
        wall = None
else:
    product_type = h1[0]
    if 'Труба' in product_type:
        try:
            diameter = float(h1[1].split('х')[0])
        except:
            diameter = None
        try:
            wall = float(h1[1].split('х')[-1].replace(',', '.'))
        except:
            wall = None
    else:
        continue
try:
    price = float(soup.find('span', {'itemprop': 'price'}).text.strip())
except:
    price = 0
try:
    max_price = float(p_data[3].strip().replace(' ', '').replace(u'\xa0', ''))
except:
    max_price = 0
try:
    min_price = float(p_data[5].strip().replace(' ', '').replace(u'\xa0', ''))
except:
    min_price = 0
if p_data[6] == 'Марка стали/ ГОСТ:':
    standard = p_data[7].split()[2]
    steel = p_data[7].split()[3].split('.')[1]
    main_list.append({'region': 'хабаровск',
                      'type': product_type,
                      'diameter': diameter,
                      'wall': wall,
                      'price': price,
                      'max_price': max_price,
                      'min_price': min_price,
                      'url': parth_url + info_href,
                      'standard': standard,
                      'steel': steel})
else:
    main_list.append({'region': 'хабаровск',
                      'type': product_type,
                      'diameter': diameter,
                      'wall': wall,
                      'price': price,
                      'max_price': max_price,
                      'min_price': min_price,
                      'url': parth_url + info_href})

In [ ]:
json_list = json.dumps(main_list,  ensure_ascii=False).replace('},', '},\n')
with open('dmk2.json', "w",  encoding='utf-8') as file:
    file.write(json_list)